In [11]:
import os
from matplotlib import pyplot as plt
import mediapipe as mp
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense
import pandas as pd
import cv2
import numpy as np
import math

In [12]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles
pose = mp_pose.Pose()

In [24]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def calculate_angles(firstPoint, midPoint, lastPoint):
  result = math.degrees(math.atan2(lastPoint.y  - midPoint.y, lastPoint.x - midPoint.x) - math.atan2(firstPoint.y - midPoint.y, firstPoint.x - midPoint.x))
  result = abs(result)
  if result > 180:
      result = 360.0 - result
  return result


def get_standardized_angle(pose_angles):
  average = sum(pose_angles) / len(pose_angles)
  return round(average)

def extract_angles(results):
  if results.pose_landmarks is None:
      return None
  
  # Calculates the essential angles for each image and adds them to the array
  left_elbow_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST] , results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER])
  right_elbow_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER])
  left_shoulder_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP])
  right_shoulder_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP])
  left_hip_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE])
  right_hip_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE])
  left_knee_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL])
  right_knee_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL])

  golfdataset = pd.DataFrame([[left_elbow_angles, right_elbow_angles, left_shoulder_angles, right_shoulder_angles, left_hip_angles, right_hip_angles, left_knee_angles, right_knee_angles, 5]], columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', "Label"])
  #golfdataset.to_csv('GolfMediaPipeData.csv', mode='a', index=False, header=False)


In [25]:
# Set up Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

min_detection_confidence = 0.5
min_tracking_confidence = 0.5

image_dir = './GolfDB_Dataset/Impact/'
image_filenames = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)
                   if os.path.isfile(os.path.join(image_dir, filename))]
image_filenames = sorted(image_filenames, key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

ctr = 0
success_ctr = 0
skipped_ctr = 0

for filename in image_filenames:

    image = cv2.imread(filename)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect the pose
    with mp_pose.Pose(min_detection_confidence=min_detection_confidence,
                      min_tracking_confidence=min_tracking_confidence) as pose:
        results = pose.process(image)

        if results.pose_landmarks is None:
            print(f"Pose landmarks not found in {filename}")
            skipped_ctr += 1
        else:
            keypoints = extract_angles(results)
            success_ctr += 1
            ctr += 1

            annotated_image = image.copy()
            mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Display the annotated image
            cv2.imshow('Pose Detection', annotated_image)

        # if ctr >= 450:
        #     break

    # if cv2.waitKey(0) & 0xFF == ord('e'):
    #     # cv2.destroyAllWindows()
    #     break

print(f"Successfully captured all pose landmarks in {success_ctr} out of {ctr + skipped_ctr} images.")
print(f"Skipped {skipped_ctr} images.")

cv2.destroyAllWindows()


Pose landmarks not found in ./GolfDB_Dataset/Impact/20.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/21.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/59.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/204.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/223.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/226.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/281.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/409.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/439.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/498.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/503.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/512.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/618.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/701.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/797.jpg
Pose landmarks not found in ./GolfDB_Dataset/Impact/822.jpg
Pose landmarks not found in ./GolfDB_Datase

In [26]:
dataset = pd.read_csv("C:\\Users\\Crim\\Desktop\\RESEARCH_THESIS\\golfpose\\GolfMediaPipeData.csv")
dataset.head()

,x1,x2,x3,x4,x5,x6,x7,x8,Label
0,173.382680,162.905673,1.682428,1.951471,157.318779,158.426863,176.730082,178.661229,1
1,165.253038,156.930396,7.233925,1.591685,153.671720,160.914156,173.217956,177.346512,1
2,161.581891,162.968698,3.535050,1.496108,169.982488,156.329636,170.023857,179.863322,1
3,169.232114,162.796697,0.613883,0.804719,173.916213,154.806477,176.045937,177.358669,1
4,173.758449,175.173183,3.802594,0.828725,162.272318,157.735769,174.503925,171.857373,1


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve

# Load the dataset
df = pd.read_csv('GolfMediaPipeData.csv' )
X = df[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']]
X = X.values
print(X)
y = df[['Label']]
y = y.values

[[173.38268005 162.90567338   1.68242825 ... 158.42686344 176.73008236
  178.6612294 ]
 [165.25303778 156.93039561   7.23392467 ... 160.91415637 173.21795634
  177.34651214]
 [161.5818915  162.96869754   3.5350505  ... 156.32963619 170.02385739
  179.86332157]
 ...
 [170.30145786 137.80700653  24.55270026 ... 167.42825367 173.14388836
  147.42581989]
 [166.25507142 148.85340556  28.3708355  ... 144.51118308 172.88904156
  169.4377574 ]
 [177.88561146 150.09127057  18.96368122 ... 137.2537705  176.26022446
  178.02589643]]


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

clf = SVC(kernel='rbf', gamma=0.1, C=1, probability=True)

clf.fit(X_scaled, y)

y_pred = clf.predict(X_scaled)

print(y_pred)
accuracy = accuracy_score(y, y_pred)
print("Mediapipe Accuracy:", accuracy)

c:\Users\Crim\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1 1 1 ... 5 5 5]
Mediapipe Accuracy: 0.8905630452022204


In [ ]:
# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = SVC(kernel='rbf', gamma=0.1, C=1, probability=True)
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)

print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
# obtain the decision function values for the test data
decision_values = clf.decision_function(X_test_scaled)

max_indices = decision_values.argmax(axis=1)
max_labels = clf.classes_[max_indices]

# print the highest confidence level for each label
for i in range(5):
    max_confidence = decision_values[:,i][max_indices == i].max()
    print("Highest confidence for label", i, ":", max_confidence)

In [ ]:
train_sizes = np.linspace(0.1, 1.0, 20)
train_sizes, train_scores, test_scores = learning_curve(
    clf, X, y, train_sizes=train_sizes, cv=5)

# calculate the mean and standard deviation of the training and validation scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# plot the learning curve
plt.plot(train_sizes, train_mean, label='Training score')
plt.plot(train_sizes, test_mean, label='Cross-validation score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.legend(loc='best')
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.title('Learning curve for SVM with RBF kernel')
plt.show()

In [ ]:
def extract_angles_test(results):
  if results.pose_landmarks is None:
      return None
  
  left_elbow_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST] , results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER])
  right_elbow_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER])
  left_shoulder_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP])
  right_shoulder_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP])
  left_hip_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE])
  right_hip_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE])
  left_knee_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE], results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL])
  right_knee_angles = calculate_angles(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE], results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL])

  return [[left_elbow_angles, right_elbow_angles, left_shoulder_angles, right_shoulder_angles, left_hip_angles, right_hip_angles, left_knee_angles, right_knee_angles]]

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Set up Mediapipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

min_detection_confidence = 0.5
min_tracking_confidence = 0.5

video_file = './8.mp4'
cap = cv2.VideoCapture(video_file)
highest_conf_frames = {}

while cap.isOpened():
    
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect the pose
    with mp_pose.Pose(min_detection_confidence=min_detection_confidence,
                      min_tracking_confidence=min_tracking_confidence) as pose:
        results = pose.process(frame)
        test_value = extract_angles_test(results)
        print(test_value)
        test_scaled_value = scaler.transform(test_value)
        test_probabilities = clf.predict_proba(test_scaled_value)
        test_prediction = np.argmax(test_probabilities)
        test_confidence = test_probabilities[0][test_prediction] * 100
        print(f"Predicted label: {test_prediction}, confidence: {test_confidence:.2f}%")

        # Check if pose_landmarks attribute is not found or does not contain all expected landmarks
        if not results.pose_landmarks or len(results.pose_landmarks.landmark) != 33:
            continue

        # Draw the pose landmarks on the image
        annotated_image = frame.copy()

        # Check if current frame has highest confidence level for predicted label
        if test_prediction not in highest_conf_frames or test_confidence > highest_conf_frames[test_prediction][0]:
            # Update dictionary with current frame as new highest confidence level frame
            highest_conf_frames[test_prediction] = (test_confidence, annotated_image)

    cv2.imshow('Pose Detection', annotated_image)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Display the highest confidence level frames for each predicted label
for label, (confidence, frame) in highest_conf_frames.items():
    print(f"Label {label} has highest confidence level of {confidence:.2f}%")
    cv2.imshow(f"Label {label}", frame)
    cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()